In [1]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [4]:
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
from tqdm.auto import tqdm
import re

openai_api_key  = get_secret('openai')['api_key']

mongodb_connect = get_secret('mongodb1')['api_key']

In [5]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
job_collection = db.job_collection # make sure the collected tweets are unique

In [9]:
filter={

    
}
project={
    'PositionID': 1, 
    'QualificationSummary': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['job_collection'].find(
  filter=filter,
  projection=project
)

In [10]:
job_data = []
for job in result:
    job_data.append({'position_id':job['PositionID'],
                     'summary':job['QualificationSummary']})

In [11]:
print('Number of jobs: ',len(job_data))

Number of jobs:  955


In [12]:
for job in tqdm(job_data):
  
    prompt = f"""
    Identify the common technology skills from the following job summary, 
    job summary: {job['summary']},
    format the items in a JSON list,
    be consistent, generalize, and be concise,
    if no technlology skills presented, use "Unknown" in the list.
    Do not wrapthe JSON codes in JSON markers

 
    """
#     print(prompt)
    try:
        extract_result =openai_help(prompt)
    #     print(sentiment_result)

        job_collection.update_one(
            {'PositionID':job['position_id']},
            {"$set":{'skills':json.loads(extract_result)}}
        )
    except:
        pass

  0%|          | 0/955 [00:00<?, ?it/s]